# Cutover Wave Plan — Agents Notebook

Paste/replace your Mermaid here for traceability:

```mermaid
flowchart LR 
  %% ===================== CUTOVER WAVE PLAN =====================
  %% PVA topics -> MAF/SK skills; dual-run matrix; deprecations

  %% -------- SOURCE (AS-IS) --------
  subgraph PVA["PVA Topics (As-Is)"]
    T_FAQ[Topic: FAQ / Hours]
    T_HR[Topic: HR Benefits]
    T_IT[Topic: IT Helpdesk]
    T_GIVE[Topic: Giving / Donations]
    T_PRAYER[Topic: Prayer Requests]
  end

  %% -------- TARGET (TO-BE) --------
  subgraph MAF["MAF / Semantic Kernel Skills (To-Be)"]
    S_GROUND[Skill: Grounded QA\n(AI Search retriever)]
    S_HR[Skill: HR Policy Lookup\n(Dataverse / SharePoint)]
    S_IT[Skill: IT Ticket Ops\n(ServiceDesk API)]
    S_PAY[Skill: Payment Link / Receipt\n(HTTP action)]
    S_FORM[Skill: Form Intake\n(Logic Apps / Forms)]
  end

  %% -------- WAVES --------
  subgraph WAVES["Cutover Waves"]
    direction TB
    W1[Wave 1\n- FAQ -> Grounded QA\n- Low risk, read-only\n- Canary 10% traffic]
    W2[Wave 2\n- HR & IT workflows\n- Auth/RBAC checks\n- Blue/Green deploy]
    W3[Wave 3\n- Donations & Forms\n- PCI/PII review\n- Full switchover]
  end

  %% -------- DUAL-RUN MATRIX --------
  subgraph DUAL["Dual-Run Matrix (shadow & compare)"]
    direction TB
    DR_FAQ[Row: FAQ\nPVA: Active | MAF: Shadow\nMetric: answer match >= 90%]
    DR_HR[Row: HR\nPVA: Active | MAF: Canary 10%\nMetric: p95 latency <= SLO]
    DR_IT[Row: IT\nPVA: Active | MAF: Shadow\nMetric: ticket success >= 99%]
    DR_PAY[Row: Giving\nPVA: Active | MAF: Off\nPrereq: PCI review complete]
    DR_FORM[Row: Forms\nPVA: Active | MAF: Shadow\nMetric: form parity 100%]
  end

  %% -------- DEPRECATIONS / EXIT CRITERIA --------
  subgraph DEPREC["Deprecations & Exit Criteria"]
    direction TB
    CRIT_QUAL[Quality gates\n- Factuality >= 0.90\n- Citation coverage >= 0.85]
    CRIT_LAT[Performance gates\n- p95 latency within SLO\n- Error rate < 0.5%]
    CRIT_COST[Cost gates\n- $/turn within budget]
    COMM[Comms: announce cutover\nrelease notes + owner sign-off]
    SWITCH[Router switch\nPVA intent -> MAF skill]
    SUNSET[Deprecate old PVA topics\narchive transcripts]
  end

  %% ===================== MAPPINGS =====================
  T_FAQ -->|Wave 1| S_GROUND
  T_HR -->|Wave 2| S_HR
  T_IT -->|Wave 2| S_IT
  T_GIVE -->|Wave 3| S_PAY
  T_PRAYER -->|Wave 3| S_FORM

  W1 -. covers .- DR_FAQ
  W2 -. covers .- DR_HR
  W2 -. covers .- DR_IT
  W3 -. covers .- DR_PAY
  W3 -. covers .- DR_FORM

  DR_FAQ --> CRIT_QUAL
  DR_HR  --> CRIT_LAT
  DR_IT  --> CRIT_QUAL
  DR_PAY --> CRIT_COST
  DR_FORM--> CRIT_QUAL

  CRIT_QUAL --> SWITCH
  CRIT_LAT  --> SWITCH
  CRIT_COST --> SWITCH
  SWITCH --> COMM --> SUNSET

  %% ===================== STYLES =====================
  classDef src fill:#e8f0fe,stroke:#1a73e8,stroke-width:2px,color:#0b468c;
  classDef tgt fill:#e8fff3,stroke:#10b981,stroke-width:2px,color:#065f46;
  classDef wave fill:#fff7ed,stroke:#fb923c,stroke-width:2px,color:#7c2d12;
  classDef dual fill:#fef9c3,stroke:#f59e0b,stroke-width:2px,color:#7c2d12;
  classDef dep fill:#f5f3ff,stroke:#8b5cf6,stroke-width:2px,color:#4c1d95;

  class PVA,T_FAQ,T_HR,T_IT,T_GIVE,T_PRAYER src
  class MAF,S_GROUND,S_HR,S_IT,S_PAY,S_FORM tgt
  class WAVES,W1,W2,W3 wave
  class DUAL,DR_FAQ,DR_HR,DR_IT,DR_PAY,DR_FORM dual
  class DEPREC,CRIT_QUAL,CRIT_LAT,CRIT_COST,COMM,SUNSET,SWITCH dep

  %% ===================== NOTES =====================
  %% - Dual-run: PVA answers live while MAF runs shadow/canary; compare nightly metrics.
  %% - Switch only after quality, latency, and cost gates pass; keep instant rollback.
  %% - Archive or hide legacy PVA topics after sunset; retain transcripts for audit.
```


In [ ]:
# %% [SETUP-ENV]
import os, getpass
os.environ.setdefault('AZURE_OPENAI_ENDPOINT', 'https://4th-openai-resource.openai.azure.com')
os.environ.setdefault('AZURE_OPENAI_DEPLOYMENT', 'gpt-35-turbo')
os.environ.setdefault('AZURE_OPENAI_API_VERSION', '2024-10-21')
if not os.getenv('AZURE_OPENAI_API_KEY'):
    os.environ['AZURE_OPENAI_API_KEY'] = getpass.getpass('Enter AZURE_OPENAI_API_KEY (hidden): ').strip()
print('Azure OpenAI env ready (key is session-only).')

In [ ]:
# %% [KERNEL]
# Initialize Semantic Kernel with Azure OpenAI chat completion
import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = Kernel()

service = AzureChatCompletion(
    service_id="azure",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # api_version can be omitted; using env default above
)

kernel.add_service(service)
print("Kernel ready (Azure OpenAI)")

In [ ]:
# %% [TOOLS]
# Minimal tool stubs that match the Skills in the diagram
def tool_grounded_qa(query:str, **kw):
    return {"tool":"grounded_qa","query":query,"note":"stub – integrate AI Search retriever here"}

def tool_hr_policy_lookup(topic:str, **kw):
    return {"tool":"hr_policy_lookup","topic":topic,"note":"stub – call Dataverse/SharePoint"}

def tool_it_ticket(action:str, **kw):
    return {"tool":"it_ticket","action":action,"note":"stub – call ServiceDesk API"}

def tool_payment_link(amount:float, **kw):
    return {"tool":"payment_link","amount":amount,"note":"stub – create payment URL/receipt via HTTP"}

def tool_form_intake(schema:str, **kw):
    return {"tool":"form_intake","schema":schema,"note":"stub – trigger Logic Apps / Forms"}

TOOLS = {
    "grounded_qa": tool_grounded_qa,
    "hr_policy_lookup": tool_hr_policy_lookup,
    "it_ticket": tool_it_ticket,
    "payment_link": tool_payment_link,
    "form_intake": tool_form_intake,
}
print("Tools:", list(TOOLS.keys()))

In [ ]:
# %% [AGENTS]
# Five capability agents aligned to target skills. Each has:
# - name
# - available tools
# - run(): LLM-backed behavior (safe if SK not configured)
from typing import Dict, Any

class BaseAgent:
    def __init__(self, kernel, name:str, tools:list[str]):
        self.kernel = kernel
        self.name = name
        self.tools = tools

    def available_tools(self):
        return [t for t in self.tools if t in TOOLS]

    def call(self, tool_name:str, **kwargs):
        fn = TOOLS.get(tool_name)
        if not fn:
            raise ValueError(f"Tool not found: {tool_name}")
        return fn(**kwargs)

    async def run(self, user_text:str) -> str:
        # keep simple; if SK missing service, report gracefully
        try:
            result = await self.kernel.invoke_prompt(f"You are the {self.name} agent. User: {user_text}")
            return str(result)
        except Exception as e:
            return f"[{self.name}] LLM not configured or failed: {e}"

class AgentGroundedQA(BaseAgent):
    def __init__(self, kernel): super().__init__(kernel, "Grounded QA", ["grounded_qa"])

class AgentHR(BaseAgent):
    def __init__(self, kernel): super().__init__(kernel, "HR Policy Lookup", ["hr_policy_lookup"])

class AgentIT(BaseAgent):
    def __init__(self, kernel): super().__init__(kernel, "IT Ticket Ops", ["it_ticket"])

class AgentPay(BaseAgent):
    def __init__(self, kernel): super().__init__(kernel, "Payment Link / Receipt", ["payment_link"])

class AgentForm(BaseAgent):
    def __init__(self, kernel): super().__init__(kernel, "Form Intake", ["form_intake"])

# Instances
agent_ground = AgentGroundedQA(kernel)
agent_hr     = AgentHR(kernel)
agent_it     = AgentIT(kernel)
agent_pay    = AgentPay(kernel)
agent_form   = AgentForm(kernel)

print("Agents:", [a.name for a in [agent_ground, agent_hr, agent_it, agent_pay, agent_form]])

In [ ]:
# %% [WIRES]
# Map PVA topics (source) -> MAF/SK agents (target), plus a simple router.
ROUTES = {
    "FAQ / Hours": "ground",
    "HR Benefits": "hr",
    "IT Helpdesk": "it",
    "Giving / Donations": "pay",
    "Prayer Requests": "form",
}

AGENT_INDEX = {
    "ground": agent_ground,
    "hr": agent_hr,
    "it": agent_it,
    "pay": agent_pay,
    "form": agent_form,
}

# quick validator: ensure each route has at least one available tool
def validate_wiring():
    problems = []
    for topic, key in ROUTES.items():
        agent = AGENT_INDEX[key]
        if not agent.available_tools():
            problems.append(f"{topic} -> {agent.name} has no available tools")
    return problems

issues = validate_wiring()
print("Wiring OK" if not issues else "Wiring issues:\n- " + "\n- ".join(issues))

In [ ]:
# %% [DEMO]
# Shadow/canary-style dry run that does *not* require the LLM to be configured.
import asyncio, time

async def demo_run():
    t0 = time.time()
    samples = [
        ("FAQ / Hours", "What time do you open on Fridays?"),
        ("HR Benefits", "How do I enroll in dental?"),
        ("IT Helpdesk", "Reset my password."),
        ("Giving / Donations", "I need a receipt for a $25 donation."),
        ("Prayer Requests", "Submit a prayer request for Sunday."),
    ]
    outputs = []
    for topic, text in samples:
        key = ROUTES[topic]
        agent = AGENT_INDEX[key]

        # tool demo (stubbed)
        tool_out = None
        if key == "ground":
            tool_out = agent.call("grounded_qa", query=text)
        elif key == "hr":
            tool_out = agent.call("hr_policy_lookup", topic="benefits")
        elif key == "it":
            tool_out = agent.call("it_ticket", action="reset_password")
        elif key == "pay":
            tool_out = agent.call("payment_link", amount=25.0)
        elif key == "form":
            tool_out = agent.call("form_intake", schema="prayer_form_v1")

        # LLM attempt (will gracefully fallback if not configured)
        llm_out = await agent.run(text)

        outputs.append({
            "topic": topic,
            "agent": agent.name,
            "tool_result": tool_out,
            "llm_result": llm_out[:220] + ("..." if len(llm_out) > 220 else "")
        })
    elapsed_ms = int((time.time() - t0)*1000)
    return {"elapsed_ms": elapsed_ms, "runs": outputs}

result = asyncio.run(demo_run())
print("Elapsed (ms):", result["elapsed_ms"])
for r in result["runs"]:
    print(f"\nTopic: {r['topic']} -> Agent: {r['agent']}\nTool: {r['tool_result']}\nLLM:  {r['llm_result']}")